# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/traitlets/traitlets.py:1013: DeprecationWarning: Widget._active_widgets is deprecated.
  value = getattr(cls, name)
/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/traitlets/traitlets.py:1013: DeprecationWarning: Widget._widget_types is deprecated.
  value = getattr(cls, name)
/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/traitlets/traitlets.py:1013: DeprecationWarning: Widget.widget_types is deprecated.
  value = getattr(cls, name)
/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/traitlets/traitlets.py:1013: DeprecationWarning: Widget.widgets is deprecated.
  value = getattr(cls, name)


### 2. Instantiate the Environment and Agent

In [2]:
env = gym.make('BipedalWalker-v3')
env.seed(10)
agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], random_seed=10)

/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/Box2D/Box2D.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(


### 3. Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [ ]:
def ddpg(n_episodes=2000, max_t=700):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 100 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 100	Average Score: -110.30	Score: -99.06
Episode 200	Average Score: -98.60	Score: -107.063
Episode 300	Average Score: -99.35	Score: -106.547
Episode 400	Average Score: -90.35	Score: -101.186
Episode 500	Average Score: -94.55	Score: -45.793
Episode 600	Average Score: -59.35	Score: -59.388
Episode 700	Average Score: -117.97	Score: -39.840
Episode 800	Average Score: -85.95	Score: -29.2479
Episode 900	Average Score: -36.36	Score: -119.14
Episode 1000	Average Score: -41.47	Score: -54.02
Episode 1057	Average Score: -52.40	Score: -47.249

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
agent.reset()   
while True:
    action = agent.act(state)
    env.render()
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break
        
env.close()

### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  